# Dogs VS Cats Classifier 🐶 🆚 🐱

## 📚 Importing libararies 

In [ ]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

## Variables

In [ ]:
image_size = 128
get_data = False # put it True at the first time
batch_size = 64
gray_scale = True

## 🔃 Loading  and preprocessing data

In [ ]:
if get_data: 
    if gray_scale:
        train_images = np.array([cv2.resize(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY), (image_size, image_size))
                             for image in glob.glob('train/*.jpg')])
        test_images  = np.array([cv2.resize(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY), (image_size, image_size))
                             for image in glob.glob('test1/*.jpg')])
    else:
        train_images = np.array([cv2.resize(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB), (image_size, image_size))
                             for image in glob.glob('train/*.jpg')])
        test_images  = np.array([cv2.resize(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB), (image_size, image_size))
                             for image in glob.glob('test1/*.jpg')])
    
    np.save("train_data0.npy", train_images)
    np.save("test_data0.npy", test_images)
else:
    train_images = np.load("train_data0.npy")
    test_images = np.load("test_data0.npy")


train_tensor = torch.from_numpy(train_images)
test_tensor = torch.from_numpy(test_images)
train_labels = torch.cat((torch.zeros((12500, 1)), torch.ones((12500, 1))), axis=0)

type(train_tensor), train_tensor.shape, train_labels.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_tensor, train_labels, test_size=.2, random_state=0)
len(X_test), len(y_train)

## Defining a custom dataset and data loaders

In [ ]:
class Dogs_and_Cats():
    def __init__(self, data, labels):
        self.samples = data
        self.labels = labels
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]


trainset = Dogs_and_Cats(X_train, y_train)
testset = Dogs_and_Cats(X_test, y_test)

In [ ]:
params = {'batch_size': batch_size,
          'shuffle': False, }

train_loader = DataLoader(trainset, **params)
test_loader = DataLoader(testset, **params)

## CNN model

In [ ]:
starting_layer = 3
if gray_scale: 
    starting_layer = 1
else:
    starting_layer = 3

class DogsVsCatsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(starting_layer, 32, 3), # 32 * 126 * 126
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 32 * 63 * 63
            
            nn.Conv2d(32, 64, 3), # 64 * 61 * 61
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 64 * 30 * 30
            
            nn.Conv2d(64, 128, 3), # 128 * 28 * 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 128 * 14 * 14
            
            nn.Conv2d(128, 256, 3), # 256 * 12 * 12
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 256 * 6 * 6
            
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 1000),
            nn.ReLU(),
            nn.Linear(1000, 200),
            nn.ReLU(),
            nn.Linear(200, 1)
        )
    
    def forward(self, input_batch):
        return torch.sigmoid(self.model(input_batch))

model = DogsVsCatsModel()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=.0001)
loss_function = nn.MSELoss()
# loss_function = nn.CrossEntropyLoss()

## 🦾 Training

In [ ]:
epochs = 10
counter = 1
for epoch in range(epochs):
    for x, y in train_loader:
        x = x.float().reshape(-1, 1, 128, 128)
        y = y.float()
        
        model.zero_grad()
        output = model(x)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        
        if counter % 100 == 0:
            print(f"batch num {counter} from {int(len(trainset)/batch_size)} with loss: {loss}")
        counter += 1
    print(f"epoch num {epoch + 1} from {epochs} with loss {loss}")
print(loss)

## 🤖 Testing 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(X_test)):
        real_class = y_test[i]
        x = X_test[i].float().view(-1, 1, 128, 128)
        net_out = model(x) 
        predicted_class = (net_out>=0.5)
        
        if predicted_class == real_class:
            correct += 1
        total += 1
        if total % 1000 == 0:
            print(f"{total} from {len(X_test)}")
        
print("Accuracy: ", round(correct/total, 3))

## 💾 Model and optimizer saving

In [ ]:
PATH = "Model.pth"
torch.save(model.state_dict(), PATH)

In [ ]:
loaded_model = DogsVsCatsModel()
loaded_model.load_state_dict(torch.load(PATH))

## 🧐 Calssification

In [ ]:
image_path = "test.jpg" # put your image path here and run the cell

if gray_scale:
    image = cv2.resize(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY), (image_size, image_size))
else:
    image = cv2.resize(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB), (image_size, image_size))

image = torch.from_numpy(image)
    
with torch.no_grad():
        image = image.float().view(-1, 1, 128, 128)
        net_out = model(image) 
        is_dog = (net_out >= 0.5)     
        is_cat = 1 - np.array(is_dog)[0][0]

if is_dog: 
    print("what a strong dog 🐶")
elif is_cat:
    print("what a beautiful cat 🐱")
    
plt.imshow(image.reshape(128, 128), cmap='gray')
plt.show()